# HyDE
For a given query, HyDE retrieval pipeline contains 4 components:
1. Promptor: bulid prompt for generator based on specific task.
2. Generator: generates hypothesis documents using Large Language Model.
3. Encoder: encode hypothesis documents to HyDE vector.
4. Searcher: search nearest neighbour for the HyDE vector (dense retrieval).

### Initialize HyDE components
We use [pyserini](https://github.com/castorini/pyserini) as the search interface.

In [22]:
import json
from pyserini.search import FaissSearcher, LuceneSearcher
from pyserini.search.faiss import AutoQueryEncoder

from hyde import Promptor, OpenAIGenerator, CohereGenerator, HyDE

In [2]:
KEY = '<api key>' # replace with your API key, it can be OpenAI api key or Cohere api key
promptor = Promptor('web search')
generator = OpenAIGenerator('text-davinci-003', KEY)
encoder = AutoQueryEncoder(encoder_dir='facebook/contriever', pooling='mean')
searcher = FaissSearcher('contriever_msmarco_index/', encoder)
corpus = LuceneSearcher.from_prebuilt_index('msmarco-v1-passage')

### Build a HyDE pipeline

In [27]:
hyde = HyDE(promptor, generator, encoder, searcher)

### Load example Query

In [37]:
query = 'how long does it take to remove wisdom tooth'

### Build Zeroshot Prompt

In [38]:
prompt = hyde.prompt(query)
print(prompt)

Please write a passage to answer the question.
Question: how long does it take to remove wisdom tooth
Passage:


### Generate Hypothesis Documents

In [39]:
hypothesis_documents = hyde.generate(query)
for i, doc in enumerate(hypothesis_documents):
    print(f'HyDE Generated Document: {i}')
    print(doc.strip())

HyDE Generated Document: 0
There is no one-size-fits-all answer to this question, as the time it takes to remove a wisdom tooth can vary depending on the individual case. In general, however, the procedure usually takes around 30 minutes to an hour to complete.
HyDE Generated Document: 1
It generally takes around 30 to 45 minutes to remove a wisdom tooth. However, the time may vary depending on the position of the tooth and the amount of work required.
HyDE Generated Document: 2
It usually takes around 30 to 45 minutes to remove a wisdom tooth. However, the length of time may vary depending on the individual case.
HyDE Generated Document: 3
It can take anywhere from a few days to a few weeks to remove a wisdom tooth. The length of time will depend on the individual case and the severity of the tooth.
HyDE Generated Document: 4
The length of time it takes to remove a wisdom tooth varies depending on the tooth's position and the amount of bone surrounding it. The procedure can take anywh

### Encode HyDE vector

In [40]:
hyde_vector = hyde.encode(query, hypothesis_documents)
print(hyde_vector.shape)

(1, 768)


### Search Relevant Documents

In [41]:
hits = hyde.search(hyde_vector, k=10)
for i, hit in enumerate(hits):
    print(f'HyDE Retrieved Document: {i}')
    print(hit.docid)
    print(json.loads(corpus.doc(hit.docid).raw())['contents'])

HyDE Retrieved Document: 0
4174313
The time it takes to remove the tooth will vary. Some procedures only take a few minutes, whereas others can take 20 minutes or longer. After your wisdom teeth have been removed, you may experience swelling and discomfort, both on the inside and outside of your mouth.This is usually worse for the first three days, but it can last for up to two weeks. Read more about how a wisdom tooth is removed and recovering from wisdom tooth removal.he time it takes to remove the tooth will vary. Some procedures only take a few minutes, whereas others can take 20 minutes or longer. After your wisdom teeth have been removed, you may experience swelling and discomfort, both on the inside and outside of your mouth.
HyDE Retrieved Document: 1
18103
Before having your wisdom teeth removed, you'll be given an injection of local anaesthetic to numb the tooth and surrounding area. If you're particularly anxious about the procedure, your dentist or surgeon may give you a se

### End to End Search

e2e search will directly go through all the steps descripted above.

In [42]:
hits = hyde.e2e_search(query, k=10)
for i, hit in enumerate(hits):
    print(f'HyDE Retrieved Document: {i}')
    print(hit.docid)
    print(json.loads(corpus.doc(hit.docid).raw())['contents'])

HyDE Retrieved Document: 0
4174313
The time it takes to remove the tooth will vary. Some procedures only take a few minutes, whereas others can take 20 minutes or longer. After your wisdom teeth have been removed, you may experience swelling and discomfort, both on the inside and outside of your mouth.This is usually worse for the first three days, but it can last for up to two weeks. Read more about how a wisdom tooth is removed and recovering from wisdom tooth removal.he time it takes to remove the tooth will vary. Some procedures only take a few minutes, whereas others can take 20 minutes or longer. After your wisdom teeth have been removed, you may experience swelling and discomfort, both on the inside and outside of your mouth.
HyDE Retrieved Document: 1
91493
The time it takes to remove the tooth will vary. Some procedures only take a few minutes, whereas others can take 20 minutes or longer. After your wisdom teeth have been removed, you may experience swelling and discomfort, b